In [ ]:
# !pip install ragas langchain langchain-openai

In [1]:
import pandas as pd

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from langchain_text_splitters import TokenTextSplitter
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document
from ragas import evaluate
from ragas.testset import TestsetGenerator
from ragas.metrics import LLMContextRecall, LLMContextPrecisionWithReference, Faithfulness, AnswerRelevancy
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

from src.utils.config import ConfigDB, ConfigLLM
from src.agent.prompts.analyst_prompt import ANALYSIS_SYSTEM_PROMPT




/Users/kim/Desktop/SKN21_3rd_4Team/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/by/sbz15vxn1xz8499rjvlm_ctm0000gn/T/ipykernel_55293/529838623.py:13: DeprecationWarning: Importing LLMContextRecall from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import LLMContextRecall
  from ragas.metrics import LLMContextRecall, LLMContextPrecisionWithReference, Faithfulness, AnswerRelevancy
/var/folders/by/sbz15vxn1xz8499rjvlm_ctm0000gn/T/ipykernel_55293/529838623.py:13: DeprecationWarning: Importing LLMContextPrecisionWithReference from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import LLMCon

In [2]:
COLLECTION_NAME = ConfigDB.COLLECTION_NAME
EMBEDDING_MODEL = ConfigDB.EMBEDDING_MODEL
VECTOR_SIZE = ConfigDB.VECTOR_SIZE
OPENAI_MODEL = ConfigLLM.OPENAI_MODEL

COLLECTION_NAME, EMBEDDING_MODEL, VECTOR_SIZE, OPENAI_MODEL

('learning_ai', 'text-embedding-3-large', 3072, 'gpt-4o-mini')

In [3]:
# 데이터셋을 생성할 때 사용할 context를 추출 - sampling
client = QdrantClient(host=ConfigDB.HOST, port=ConfigDB.PORT)

# 전체 데이터를 다 조회해서 그 중 랜덤하게 K개만 sampling
info = client.get_collection(COLLECTION_NAME)
total_count = info.points_count

print("info : ", info)
print("total_count : ", total_count)

info :  status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> warnings=None indexed_vectors_count=15315 points_count=15315 segments_count=4 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=3072, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None, inline_storage=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=10000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahe

In [4]:
results, next_id = client.scroll(
    collection_name=COLLECTION_NAME,
    limit=info.points_count,
)

# sampling
import random
sample_dataset = random.sample(results, 5) # 리스트에서 랜덤하게 K개를 추출


# 문서 내용만 추출
docs = [point.payload['page_content'] for point in sample_dataset]
docs

['[KEYWORDS] __init__ voltage x\n[TITLE] functions\n[H1] Built-in Functions\n\nclass Parrot:\n          def __init__(self):\n              self._voltage = 100000\n\n          @property\n          def voltage(self):\n              """Get the current voltage."""\n              return self._voltage\n\n   The ``@property`` decorator turns the voltage method into a "getter"\n   for a read-only attribute with the same name, and it sets the docstring for\n   *voltage* to "Get the current voltage."\n\n      A property object has ``getter``, ``setter``,\n      and ``deleter`` methods usable as decorators that create a\n      copy of the property with the corresponding accessor function set to the\n      decorated function.  This is best explained with an example:\n\n         class C:\n             def __init__(self):\n                 self._x = None\n\n             @property\n             def x(self):\n                 """I\'m the \'x\' property."""\n                 return self._x',
 '[KEYWORD

In [5]:
#################################################################
# Vector DB 연결
# retriever 생성
#################################################################

def get_vectorstore(collection_name: str):
    # 1. 임베딩 모델 설정
    dense_embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)
    # 2. Qdrant 클라이언트 연결
    client = QdrantClient(host="localhost", port=6333)
    # 3. 기존 컬렉션에 연결
    vectorstore = QdrantVectorStore(
        client=client,
        collection_name=collection_name,
        embedding=dense_embeddings
    )
    
    return vectorstore


def get_retriever(vectorstore, k: int = 5):
    retriever = vectorstore.as_retriever(
        search_kwargs={"k": k}
    )
    return retriever

In [6]:
vectorstore = get_vectorstore(COLLECTION_NAME)
retriever = get_retriever(vectorstore)

In [7]:
################################################################################
# 평가할 RAG Chain
################################################################################
prompt_txt = ANALYSIS_SYSTEM_PROMPT
prompt = ChatPromptTemplate.from_template(
    template=prompt_txt
)


def format_docs(documents:list)->str:
    """
    VectorStore에 조회한 문서들에서 내용(page_content)만 추출해서 str으로 합쳐서 반환.
    VectorStore의 검색결과인 List[Document]를 받아서 Document들에서 page_content의 내용만 추출한다.
    
    Args:
        documents(list[Document]): [Document(..), Document(...), ..]}
    Returns:
        str: 각 문서의 내용을 "\n\n"으로 연결한 string
    """
    return "\n\n".join(doc.page_content for doc in documents)

model = ChatOpenAI(model=OPENAI_MODEL)
parser = StrOutputParser()

# RAG 평가를 위해서 "답변", "검색한 문서" 둘이 출력되도록 변경.
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from operator import itemgetter

def format_doc_list(docs_dict: dict) -> list:
    # dictionary[list[Document], query:str] -> list[str]  
    # 문서내용만 추출해서(Document.page_content)만 추출한 리스트
    return [doc.page_content for doc in docs_dict['context']]

# dict | dict -> dict
# RunnablePassthrough() | dict | dict ==> RunnableSequence
chain = RunnablePassthrough() | {
    "context":retriever,
    "query":RunnablePassthrough()
} | {
    "response": prompt | model | parser,
    "retrieved_context": format_doc_list, # RAGAS 평가시 context -> List[str]
}

In [26]:
res = chain.invoke("파인튜닝의 장점은 무엇인가?")
res

{'response': "파인튜닝의 장점은 사전 학습된 모델(Foundation 모델)을 특정 태스크나 도메인 데이터에 맞게 최적화하여 성능을 극대화하는 것입니다. 이를 통해 모델은 특정 작업이나 도메인에 대한 전문성을 갖추게 되어 더욱 효율적으로 동작할 수 있습니다.\n\n강의 자료에서 이와 관련된 내용은 '파인튜닝(Fine Tuning)의 필요성' 섹션에 자세히 설명되어 있으며, 파인튜닝을 통해 의료 전문 AI 모델이나 사내 특화 QA 모델을 만드는 예시가 나와 있습니다. 이러한 과정은 모델의 성능을 높이고 사용자 맞춤형 톤앤매너를 적용하는 데에도 기여합니다. \n\n한 줄로 정리하자면, 파인튜닝은 모델을 특정 도메인에 맞춰 최적화하여 성능을 개선하는 과정입니다.",
 'retrieved_context': ['[강의: 1_LLM_Finetuning개요]\n\n파인튜닝(Fine Tuning)의 필요성\n파인튜닝은 사전 학습된 모델(Foundation 모델)을 **특정 태스크나 도메인 데이터로 추가 학습 최적화 하는 과정**이다.  \n- **특정 작업/도메인에 최적화**\n- 파운데이션 모델은 범용적인 언어 이해와 생성 능력을 가지지만, 특정 도메인(업무)이나 작업에 맞춰져 있지 않다.\n- 파인튜닝은 특정 목적에 맞게 모델을 조정하여 성능을 극대화 할 수 있다.\n- 예시\n- 파운데이션 모델에 **의료 문서, 진단 기록**등으로 추가 학습하면 **의료 전문 AI 모델**을 만들 수 있다.\n- 회사에서 LLM을 사용할 경우, 범용 모델은 그 회사의 내부 데이터(업무 문서, 매뉴얼, 정책 등)를 학습하지 못했기 때문에 해당 내용에 대한 지식이 없다. 이러한 자료들로 파인튜닝하여 **사내 특화 QA 모델**을 만들 수 있다.  \n- **사용자 맞춤형 톤앤매너(Tone & Manner) 적용**\n- 특정 브랜드의 페르소나나 고객 응대 매뉴얼을 학습시켜, 조직이 원하는 **일관된 언어 스타일과 답변 형식을 제공**한다.  \n- **안전성과 윤리 강화**\n

In [8]:
llm_context = """당신은 Python 프로그래밍 교육 과정의 **AI 조교(Teaching Assistant)**이자 데이터셋 생성 전문가입니다.
제공된 [강의 자료(Lecture)]와 [Python 공식 문서(RST)]를 기반으로, 학습자 평가를 위한 고품질의 JSON 포맷 질의응답(QA) 데이터셋을 생성하십시오.

생성 시 다음의 **지침(Instruction)과 답변 스타일(Tone & Style)**을 철저히 준수해야 합니다.

---

## Instruction
당신은 실제 강의를 보조하는 **AI 조교(Teaching Assistant) Agent**이다.  
search agent가 제공한 **강의 자료(context)** 를 바탕으로,  
수강생의 질문에 대해 **강의 흐름 안에서 이해를 돕는 답변**을 제공한다.

---

## 기본 원칙

1. **질문 중심**
   - 사용자의 질문 의도를 먼저 파악하고, 그 질문에 직접적으로 필요한 내용만 답변한다.
   - 질문에 포함되지 않은 개념 확장이나 일반적인 배경 설명은 하지 않는다.

2. **강의 맥락 우선**
   - 모든 설명은 제공된 강의 자료(context)의 내용과 흐름을 기준으로 한다.
   - 내부 강의 자료([Original])를 최우선으로 사용하며, 외부 자료([External Web])가 있는 경우 보조적으로만 참고한다.

3. **불필요한 정보 배제**
   - 질문과 직접적인 관련이 없는 내용은 포함하지 않는다.
   - 설명이 가능하더라도 질문 범위를 벗어난다면 생략한다.

4. **정직한 한계 표현**
   - 질문에 대한 직접적인 근거가 강의 자료에 없는 경우,
     “강의 자료에서는 해당 내용이 직접적으로 다뤄지지 않는다”고 명확히 말한다.
   - 다만, 강의 흐름상 최소한의 맥락 설명이 필요한 경우에만 짧게 보충한다.

---

## 답변 방식

- 먼저 **질문에 대한 핵심 답**을 간단히 제시한다.
- 이후 필요할 경우에만:
  - 강의 자료의 어느 부분과 연결되는지
  - 코드나 실습이 있다면 그 목적과 역할
  을 설명한다.
- 질문이 단순한 경우에는 추가 설명 없이 핵심 답변만 제공해도 된다.

---

## Output Guide (유연 적용)

필요한 경우에만 아래 요소를 포함한다:

- **핵심 답변**: 질문에 대한 직접 관련된 핵심 개념을 간단히 제시한다.
- **강의 맥락 설명**: 강의 자료에서 어떤 파일 / 어떤 코드 또는 셀과 연결되는지 명확히 설명한다.
- **실습/코드 연결**: 왜 이 코드가 등장했는지에 대한 설명을 수업 흐름 기준으로 설명한다.
- **한 줄 정리**: 시험 대비 또는 복습용으로 한 문장으로 요약한다.

모든 요소를 항상 포함할 필요는 없다.

---

## Tone & Style

- 실제 강의 조교처럼 **차분하고 설명 중심적인 말투**를 사용한다.
- 과장, 불필요한 비유, 감정 표현은 사용하지 않는다.
- 질문에 대한 답변이 자연스럽게 끝나면 추가 문장은 붙이지 않는다.

---

## QA 생성 제약 (Constraints)
- **언어**: 가능하면 한국어로 질문하고 답변하되, 코드 예제는 영어로 작성하십시오.
- **문법**: 문장의 끝은 마침표(.) 등 **구두점을 반드시 표기**하여 완결된 문장으로 작성하십시오.
- **JSON 포맷 엄수**:
  - 출력은 반드시 파싱 가능한 **JSON 배열** 형태여야 합니다 (`[{"user_input": "...", "reference": "..."}, ...]`).
  - **이스케이프 처리**: 본문 내의 큰따옴표("), 백슬래시(\) 등 특수 문자는 반드시 역슬래시(\")를 사용하여 이스케이프 처리하십시오.
  - 소스 텍스트에 JSON 문법을 해치는 요소가 있더라도, 최종 출력은 유효한 JSON이어야 합니다.

---

## QA 생성 출력 예시
[
  {
    "user_input": "파인튜닝의 장점은 무엇인가?",
    "reference": "파인튜닝은 사전 학습된 모델(Foundation 모델)을 특정 태스크나 도메인 데이터로 추가 학습 최적화 하는 과정으로, 특정 작업/도메인에 최적화, 사용자 맞춤형 톤앤매너 적용, 안전성과 윤리 강화를 위해 필요하다."
  }
]

위 가이드를 바탕으로 주어진 텍스트에서 학습 가치가 높은 QA 세트를 생성하십시오.
"""

<>:73: SyntaxWarning: invalid escape sequence '\)'
<>:73: SyntaxWarning: invalid escape sequence '\)'
/var/folders/by/sbz15vxn1xz8499rjvlm_ctm0000gn/T/ipykernel_55293/2424486145.py:73: SyntaxWarning: invalid escape sequence '\)'
  - **이스케이프 처리**: 본문 내의 큰따옴표("), 백슬래시(\) 등 특수 문자는 반드시 역슬래시(\")를 사용하여 이스케이프 처리하십시오.


In [9]:
generator_llm = LangchainLLMWrapper(ChatOpenAI(model=OPENAI_MODEL))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model=EMBEDDING_MODEL))

generator = TestsetGenerator(
    llm=generator_llm,
    embedding_model=generator_embeddings,
    llm_context=llm_context 
)

# testset
testset = generator.generate_with_chunks(
    docs, testset_size=20  # context 내용, 테스트데이터셋 몇개를 만들지.
)
testset

/var/folders/by/sbz15vxn1xz8499rjvlm_ctm0000gn/T/ipykernel_55293/496705960.py:1: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  generator_llm = LangchainLLMWrapper(ChatOpenAI(model=OPENAI_MODEL))
/var/folders/by/sbz15vxn1xz8499rjvlm_ctm0000gn/T/ipykernel_55293/496705960.py:2: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model=EMBEDDING_MODEL))
Applying CustomNodeFilter:   0%|          | 0/5 [00:00<?, ?it/s]Node bc760521-816c-46c2-828d-298a4f796933 does 

Testset(samples=[TestsetSample(eval_sample=SingleTurnSample(user_input='What is the role of the comma in Python programming?', retrieved_contexts=None, reference_contexts=['[KEYWORDS] operators\n[TITLE] programming\n[H1] Core Language\n[H2] What\'s up with the comma operator\'s precedence?\n\nComma is not an operator in Python. Consider this session::\n\n    >>> "a" in "b", "a"\n    (False, \'a\')\n\nSince the comma is not an operator, but a separator between expressions the\nabove is evaluated as if you had entered::\n\n    ("a" in "b"), "a"\n\nnot::\n\n    "a" in ("b", "a")\n\nThe same is true of the various assignment operators (``=``, ``+=`` etc). They\nare not truly operators but syntactic delimiters in assignment statements.'], retrieved_context_ids=None, reference_context_ids=None, response=None, multi_responses=None, reference='In Python, the comma is not an operator but a separator between expressions. For example, in the expression "a" in "b", "a", it is evaluated as if you h

In [10]:
eval_df = testset.to_pandas()
eval_df.head()

,user_input,reference_contexts,reference,persona_name,query_style,query_length,synthesizer_name
0,What is the role of the comma in Python progra...,[[KEYWORDS] operators\n[TITLE] programming\n[H...,"In Python, the comma is not an operator but a ...",Python Developer,MISSPELLED,MEDIUM,single_hop_specific_query_synthesizer
1,What is the role of the comma in Python progra...,[[KEYWORDS] operators\n[TITLE] programming\n[H...,"In Python, the comma is not an operator but a ...",Data Visualization Analyst,MISSPELLED,MEDIUM,single_hop_specific_query_synthesizer
2,"Pythn deosn't have a comma operator, but what ...",[[KEYWORDS] operators\n[TITLE] programming\n[H...,"In Python, the comma is not an operator but a ...",Data Visualization Analyst,MISSPELLED,LONG,single_hop_specific_query_synthesizer
3,Is the comma an operator in Python?,[[KEYWORDS] operators\n[TITLE] programming\n[H...,"No, the comma is not an operator in Python. It...",Data Visualization Analyst,WEB_SEARCH_LIKE,SHORT,single_hop_specific_query_synthesizer
4,Can you explain how the Timer method in the ti...,[[KEYWORDS] timeit method Timer timeit\n[TITLE...,The Timer method in the timeit module is used ...,Python Developer,PERFECT_GRAMMAR,LONG,single_hop_specific_query_synthesizer


In [11]:
# >>>> Chain 응답들을 저장할 list
response_list = []
# Chain 이 반환한 context들을 지정할 list
retrieved_context_list = []

for user_input in eval_df['user_input']:
    res = chain.invoke(user_input)
    response_list.append(res['response'])
    retrieved_context_list.append(res['retrieved_context'])

# >>>> eval_df에 응답과 context 추가
eval_df['response'] = response_list
eval_df['retrieved_contexts'] = retrieved_context_list
eval_df.head()

,user_input,reference_contexts,reference,persona_name,query_style,query_length,synthesizer_name,response,retrieved_contexts
0,What is the role of the comma in Python progra...,[[KEYWORDS] operators\n[TITLE] programming\n[H...,"In Python, the comma is not an operator but a ...",Python Developer,MISSPELLED,MEDIUM,single_hop_specific_query_synthesizer,"**핵심 답변**: Python에서 쉼표는 주로 표현식의 구분자로 사용되며, 표현식...",[[KEYWORDS] operators\n[TITLE] programming\n[H...
1,What is the role of the comma in Python progra...,[[KEYWORDS] operators\n[TITLE] programming\n[H...,"In Python, the comma is not an operator but a ...",Data Visualization Analyst,MISSPELLED,MEDIUM,single_hop_specific_query_synthesizer,"핵심 답변: Python에서 쉼표(comma)는 표현식 간의 구분자로 사용되며, 튜...",[[KEYWORDS] operators\n[TITLE] programming\n[H...
2,"Pythn deosn't have a comma operator, but what ...",[[KEYWORDS] operators\n[TITLE] programming\n[H...,"In Python, the comma is not an operator but a ...",Data Visualization Analyst,MISSPELLED,LONG,single_hop_specific_query_synthesizer,핵심 답변: Python에서 쉼표는 연산자가 아닌 표현식 간의 구분자입니다. 따라서...,[[KEYWORDS] operators\n[TITLE] programming\n[H...
3,Is the comma an operator in Python?,[[KEYWORDS] operators\n[TITLE] programming\n[H...,"No, the comma is not an operator in Python. It...",Data Visualization Analyst,WEB_SEARCH_LIKE,SHORT,single_hop_specific_query_synthesizer,콤마는 Python에서 운영자가 아닙니다. 콤마는 표현식 간의 구분자로 사용됩니다....,[[KEYWORDS] operators\n[TITLE] programming\n[H...
4,Can you explain how the Timer method in the ti...,[[KEYWORDS] timeit method Timer timeit\n[TITLE...,The Timer method in the timeit module is used ...,Python Developer,PERFECT_GRAMMAR,LONG,single_hop_specific_query_synthesizer,"핵심 답변: `Timer` 클래스는 코드 실행 시간을 측정하기 위해 사용되며, 기본...",[[KEYWORDS] timeit repeat\n[TITLE] timeit\n[H1...


In [13]:
from ragas import EvaluationDataset
# from_xxxx() xxxx 타입의 객체를 EvaluationDataset객체로 변환.
eval_dataset = EvaluationDataset.from_pandas(
    eval_df[["user_input", "retrieved_contexts", "response", "reference"]]
)
eval_dataset

EvaluationDataset(features=['user_input', 'retrieved_contexts', 'response', 'reference'], len=12)

In [14]:
eval_llm = LangchainLLMWrapper(ChatOpenAI(model=OPENAI_MODEL))
eval_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model=EMBEDDING_MODEL))
# 평가할 함수들을 List로 묶어준다.
metrics = [
    LLMContextRecall(llm=eval_llm),
    LLMContextPrecisionWithReference(llm=eval_llm),
    Faithfulness(llm=eval_llm),
    AnswerRelevancy(llm=eval_llm, embeddings=eval_embeddings)
]

# Run Evaluation
eval_results = evaluate(dataset=eval_dataset, metrics=metrics)

# Convert to Pandas DataFrame for easier viewing
df_results = eval_results.to_pandas()
display(df_results.head(10))

/var/folders/by/sbz15vxn1xz8499rjvlm_ctm0000gn/T/ipykernel_55293/3596288186.py:1: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  eval_llm = LangchainLLMWrapper(ChatOpenAI(model=OPENAI_MODEL))
/var/folders/by/sbz15vxn1xz8499rjvlm_ctm0000gn/T/ipykernel_55293/3596288186.py:2: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  eval_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model=EMBEDDING_MODEL))
Evaluating:  15%|█▍        | 7/48 [00:24<01:56,  2.84s/it]LLM returned 1 generations instead of requested 3. Proceeding

,user_input,retrieved_contexts,response,reference,context_recall,llm_context_precision_with_reference,faithfulness,answer_relevancy
0,What is the role of the comma in Python progra...,[[KEYWORDS] operators\n[TITLE] programming\n[H...,"**핵심 답변**: Python에서 쉼표는 주로 표현식의 구분자로 사용되며, 표현식...","In Python, the comma is not an operator but a ...",1.0,0.700000,0.625000,0.952747
1,What is the role of the comma in Python progra...,[[KEYWORDS] operators\n[TITLE] programming\n[H...,"핵심 답변: Python에서 쉼표(comma)는 표현식 간의 구분자로 사용되며, 튜...","In Python, the comma is not an operator but a ...",1.0,0.700000,1.000000,0.960450
2,"Pythn deosn't have a comma operator, but what ...",[[KEYWORDS] operators\n[TITLE] programming\n[H...,핵심 답변: Python에서 쉼표는 연산자가 아닌 표현식 간의 구분자입니다. 따라서...,"In Python, the comma is not an operator but a ...",1.0,1.000000,1.000000,0.712046
3,Is the comma an operator in Python?,[[KEYWORDS] operators\n[TITLE] programming\n[H...,콤마는 Python에서 운영자가 아닙니다. 콤마는 표현식 간의 구분자로 사용됩니다....,"No, the comma is not an operator in Python. It...",1.0,0.755556,1.000000,0.770319
4,Can you explain how the Timer method in the ti...,[[KEYWORDS] timeit repeat\n[TITLE] timeit\n[H1...,"핵심 답변: `Timer` 클래스는 코드 실행 시간을 측정하기 위해 사용되며, 기본...",The Timer method in the timeit module is used ...,0.8,0.750000,0.888889,0.598855
5,What is the purpose of the Timer method in the...,[[KEYWORDS] timeit repeat\n[TITLE] timeit\n[H1...,핵심 답변: `Timer` 메소드는 `timeit` 모듈에서 작은 코드 조각의 실행...,The Timer.timeit method is used to time a spec...,1.0,0.887500,0.777778,0.964633
6,What does the timeit method do in Python?,[[KEYWORDS] power operator\n[TITLE] timeit\n[H...,**핵심 답변**: `timeit` 메서드는 파이썬에서 작은 코드 조각의 실행 시간...,The timeit method measures the execution time ...,0.5,0.679167,0.833333,0.870147
7,What is the purpose of the timeit method in Py...,[[KEYWORDS] power operator\n[TITLE] timeit\n[H...,**핵심 답변**: Python의 `timeit` 메서드는 작은 코드 조각의 실행 ...,The timeit method is used to measure the execu...,1.0,1.000000,1.000000,0.921719
8,wat is catch_warnings in python?,[[KEYWORDS] warning fxn warn method warnings w...,**핵심 답변**: `catch_warnings`는 Python에서 경고를 제어하기...,The catch_warnings context manager is used for...,0.5,0.916667,1.000000,0.793917
9,What does simplefilter do in the context of wa...,[[KEYWORDS] __main__ simplefilter method warni...,**핵심 답변**: `simplefilter`는 경고 메시지에 대한 필터를 설정하는...,The simplefilter function is used to control w...,1.0,0.804167,0.875000,0.642312


- **Retrieval**
    -  질문에 대해 검색한 문서(context)들에 대한 평가
    -  **Context Precision(문맥 정밀도)**
        -  검색된 문서(context)들 중 질문과 관련 있는 것들이 **얼마나 상위 순위에 위치하는지** 평가하는 지표.
    -  **Context Recall(문맥 재현률)**
        -  검색된 문서(context)가 정답(ground-truth)의 정보를 얼마나 포함하고 있는지 평가하는 지표.
- **Generation**
    - llm 모델이 생성한 답변에 대한 평가 지표들.
    - **Faithfulness(신뢰성)**
        -  생성된 답변과 검색된 문서(context)간의 관련성을 평가하는 지표
        -  생성된 답변이 주어진 문맥(context)에 얼마나 충실한지를 평가하는 지표로 할루시네이션에 대한 평가로 볼 수있다.
    - **Answer relevancy(답변 적합성)**
        - 생성된 답변과 사용자의 질문간의 관련성을 평가하는 지표
        - 생성된 답변이 사용자의 질문과 얼마나 관련성이 있는지를 평가하는 지표.

In [25]:
print("\n📊 Evaluation Results:")
print(eval_results)


📊 Evaluation Results:
{'context_recall': 0.8583, 'llm_context_precision_with_reference': 0.8494, 'faithfulness': 0.8542, 'answer_relevancy': 0.8352}


In [27]:
eval_results.to_pandas()
eval_results

{'context_recall': 0.8583, 'llm_context_precision_with_reference': 0.8494, 'faithfulness': 0.8542, 'answer_relevancy': 0.8352}